In [26]:
from datasets import Dataset
from transformers import AutoTokenizer
import os
import random
from sklearn.model_selection import train_test_split
import gc
import torch
from datetime import datetime
import awswrangler as wr
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None

from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import DataCollatorWithPadding

import warnings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 500)
pd.options.mode.chained_assignment = None
warnings.simplefilter(action='ignore', category=FutureWarning)

In [15]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Today I'm going to use {device.type}")

Today I'm going to use cuda


In [16]:
SEED = 1234
N_SAMPLES = 20_000
TODAY = datetime.today().strftime("%Y%m%d")
BUCKET_NAME = 'sagemaker-godeltech'
TRAIN_PATH = f"s3://{BUCKET_NAME}/data/train/train.csv"
VAL_PATH = f"s3://{BUCKET_NAME}/data/validate/validate.csv"
TEST_PATH = f"s3://{BUCKET_NAME}/data/test/test.csv"
MODEL_PATH = "local_transformers/models"

In [18]:
def seed_everything(seed=SEED):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything()
gc.collect()
torch.cuda.empty_cache()
# del model
# del Trainer
# del tokenizer

In [19]:
train = wr.s3.read_csv([TRAIN_PATH])
val = wr.s3.read_csv([VAL_PATH])
test = wr.s3.read_csv([TEST_PATH])

In [20]:
train_sample = train.sample(N_SAMPLES, random_state=SEED, ignore_index=True)
val_sample = val.sample(N_SAMPLES, random_state=SEED, ignore_index=True)
train.shape, val.shape, test.shape

((1443900, 2), (360975, 2), (194641, 12))

In [21]:
train_sample['toxicity'] = train_sample['toxicity'].astype('int')
val_sample['toxicity'] = val_sample['toxicity'].astype('int')

In [22]:
# load tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased", cache_dir = '../tmp/AutoTokenizer');

# create tokenization function
def tokenize(batch):
    return tokenizer(batch["comment_text"], padding="max_length", truncation=True)

# tokenize train and test datasets
train_dataset = Dataset.from_pandas(train_sample).map(tokenize, batched=True)
val_dataset = Dataset.from_pandas(val_sample).map(tokenize, batched=True)

# set dataset format for PyTorch
train_dataset =  train_dataset.rename_column("toxicity", "labels")
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
val_dataset = val_dataset.rename_column("toxicity", "labels")
val_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at ../tmp/AutoTokenizer/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.21.3",
  "vocab_size": 30522
}

loading file https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt from cache at ../tmp/AutoTokenizer/0e1bbfda7f63a99bb52e3915dcf10c3c92122b827d92eb2d34ce94ee79ba486

  0%|          | 0/20 [00:00<?, ?ba/s]

  0%|          | 0/20 [00:00<?, ?ba/s]

In [23]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2, cache_dir = '../tmp/AutoModel')

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at ../tmp/AutoModel/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.21.3",
  "vocab_size": 30522
}

loading weights file https://huggingface.co/distilbert-base-uncased/resolve/main/pytorch_model.bin from cache at ../tmp/AutoModel/9c169103d7e5a73936dd2b627e42851bec0831212b677c637033ee4

In [27]:
training_args = TrainingArguments(
    output_dir="../tmp/results",
    logging_dir="../tmp/results/logs",
    evaluation_strategy = "steps",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=1,
    weight_decay=0.01,
    seed=SEED,
    save_total_limit = 5, # Only last 5 models are saved. Older ones are deleted.
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()

using `logging_steps` to initialize `eval_steps` to 500
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: comment_text. If comment_text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 20000
  Num Epochs = 1
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 625


Step,Training Loss,Validation Loss
500,0.189200,0.141647


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: comment_text. If comment_text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 20000
  Batch size = 32
Saving model checkpoint to ../tmp/results/checkpoint-500
Configuration saved in ../tmp/results/checkpoint-500/config.json
Model weights saved in ../tmp/results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ../tmp/results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ../tmp/results/checkpoint-500/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ../tmp/results/checkpoint-500 (score: 0.14164744317531586).


TrainOutput(global_step=625, training_loss=0.18166436462402344, metrics={'train_runtime': 1257.6677, 'train_samples_per_second': 15.902, 'train_steps_per_second': 0.497, 'total_flos': 2649347973120000.0, 'train_loss': 0.18166436462402344, 'epoch': 1.0})

In [33]:
test_text = test[['comment_text', 'toxicity']][:20000]
test_text['toxicity'] = test_text['toxicity'].astype('int')

In [34]:
# tokenize train and test datasets
test_dataset = Dataset.from_pandas(test_text).map(tokenize, batched=True)

# set dataset format for PyTorch
test_dataset.set_format("torch", columns=["input_ids", "attention_mask"])

  0%|          | 0/20 [00:00<?, ?ba/s]

In [35]:
outputs = trainer.predict(test_dataset)

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: toxicity, comment_text. If toxicity, comment_text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 20000
  Batch size = 32


In [36]:
from utils import save_to_s3

y_pred = outputs.predictions.argmax(1)
np.savetxt(f"../tmp/transformers_predictions{TODAY}.csv", y_pred, delimiter=",")
save_to_s3(BUCKET_NAME, f"../tmp/transformers_predictions{TODAY}.csv", f"{MODEL_PATH}/transformers_predictions{TODAY}.csv")

In [37]:
test_true = test[:20000]

In [38]:
from quality_calculator import compute_bias_metrics_for_model, calculate_overall_auc, get_final_metric


oof_name = 'predicted_target'
identity_columns = ['male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish', 'muslim', 'black', 'white', 'psychiatric_or_mental_illness']
test_true[oof_name] = y_pred
#evaluation
bias_metrics_df = compute_bias_metrics_for_model(test_true, identity_columns, oof_name, 'toxicity')
display(bias_metrics_df)
FINAL_SCORE = get_final_metric(bias_metrics_df, calculate_overall_auc(test_true, oof_name))
print(f"FINAL SCORE FOR CUSTOM TRANSFORMERS IS {FINAL_SCORE}")   

,subgroup,subgroup_size,subgroup_auc,bpsn_auc,bnsp_auc
8,psychiatric_or_mental_illness,19,0.485294,0.633406,0.727205
0,male,104,0.635417,0.675318,0.835912
3,christian,39,0.650000,0.716550,0.808632
2,homosexual_gay_or_lesbian,25,0.650000,0.783461,0.741909
7,white,103,0.685385,0.702991,0.857517
1,female,97,0.707393,0.789521,0.794620
6,black,65,0.797956,0.841667,0.831529
5,muslim,45,0.802632,0.883610,0.794511
4,jewish,19,0.821429,0.882804,0.813309


FINAL SCORE FOR CUSTOM TRANSFORMERS IS 0.7614367475560458
